In [98]:
# importing libarries
import pandas as pd
import numpy as np
import seaborn as sns
import re
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
import matplotlib.pyplot as plt
import matplotlib

In [99]:
df=pd.read_csv("../input/banglorehousepricedataset/Bengaluru_House_Data.csv")
df.head()

In [100]:
#data description
print(df.shape)
print(df.describe())
print(df.info())

Just by looking at the describe we can see our price is highly skewed,where the mean is 112 and  the 50% is on 72

In [101]:
#def find_number(text):
    #num = re.findall(r'[0-9]+',text)
    #return " ".join(num)
#df['no of rooms']=df['size'].apply(lambda x: find_number(x))

<a id='2'></a> <h1 style="background-color:grey; color:blue" ><br>Exploratery data analysis<br></h1>
<a id='3'></a>

<b>
*  Steps for EDA:
  
   1. Checking the missing value
   2. checking the description,datatypes
   3. Doing Univariate analysis on the variables to understand how the variables are affecting the outcome
   4. Finding  the correlation between the variables
    </b></center>
   

In [102]:

# checking the datatypes of all the  features in the dataset
print(df.select_dtypes('object').shape)
print('********')
print(df.select_dtypes('int64').shape)
print('********')
print(df.select_dtypes('float').shape)
print('********')
print(df.select_dtypes('bool').shape)

In [103]:
#checking the missing value
def miss_val(df):
   miss_col=df[[col for col in df.columns if df[col].isnull().any() == True]].isna().sum()
   return miss_col

print(miss_val(df))

1>We can see here we have almost 50% of recordis in scocitey columns which are missing.we may get rid of this column .

2>The bedroom in size has to be replaced by BHK ,but before doing that we need to check if there are other names used for BHK.We need to extract only the numbers from this column.

3> we need to do hot encoding for the location column.

4> Area_type needs to  be coded in numbers.

5> we have 73 bath as null,which is not possible we need to impudate with using the size of the hous and average bath in those houses in other records.

6> we will replace the balconey na values with median and drop all the other NA records



In [104]:
#extracting number of rooms and 
#df['size'] = df['size'].str[:2]
#df.head(2)

In [105]:
#df.size=df.size.astype(int)
df['size'].unique()

In [106]:
#handling balcony missing vaule in bacony using simple imputer
Imp = SimpleImputer(missing_values=np.NaN, strategy='median')
df.balcony=pd.DataFrame(Imp.fit_transform(df['balcony'].values.reshape(-1,1))[:,0])

#values.reshap means that you are not sure about what number of rows or columns you can give and you are asking numpy
#to suggest a number of column or rows to get reshaped in.
df

so using simple imputer will work for us and also there is not much of a deviation in the data  'Balcony' .Again there are multiple ways to do this but here we are using simpleimputer.

In [107]:
#drop column socitey and all the other missing values 

df=df.drop(columns=['society'])
df=df.dropna()

### Understanding the data

In [108]:
#encoding the cateogrical variables


df['availability']=np.where(df['availability']=='Ready To Move',1,0)

In [109]:
# handling range values in total_sqft
# we will first extract all those records which has "-" in it and then extract them and then  find the average of those numbers.

def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [110]:
df[~df['total_sqft'].apply(is_float)].head(10)

we can see we have total _sqft in  sq. meter and perch,for now we will park those values aside and focus on the range average

In [111]:
def convert_sqft_to_num(x):
    tokens=x.split('-')
    if len(tokens) == 2: 
        return(float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        #return x--> we will use this for future conversion of sqmt and yard and acres
        return None

In [112]:
df1=df.copy()
df1['total_sqft']=df1['total_sqft'].apply(convert_sqft_to_num)
df1.head()

In [113]:
# price per sqft
df2=df1.copy()
df2['ppsqft']=df2['price']*100000/df2['total_sqft']
df2.head()

In [114]:
#handling location column

en_label = preprocessing.LabelEncoder()

#for i in df.select_dtypes('object'):
#df['location']= en_label.fit_transform(df['location'])
# curse of dimensionality so if we are doing one hot encoding or label enoding it won't be good ,we will group the values here
df3=df2.copy()
df3['location']=df3['location'].apply(lambda x: x.strip())
location_stats=df3.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_stats_lessthan10=(location_stats[location_stats<=10])#grouping the location which has less than 10 hits
location_stats_lessthan10

In [115]:
len(df3.location.unique())

In [116]:
#putting any tging with less than 10 location count as others
df3.location=df3.location.apply(lambda x: 'other' if x in  location_stats_lessthan10 else x)#this will leave us with only 242 location

In [117]:
df.head()

In [118]:
##extracting number of rooms and 
df4=df3.copy()
def extract_num(x):
    tokens=x.split(' ')
    if len(tokens) == 2: 
        return(float(tokens[0]))
    try:
        return float(x)
    except:
        #return x--> we will use this for future conversion of sqmt and yard and acres
        return None
    
df4.size=df4['size'].apply(extract_num)

### Outlier detection 

In [119]:
df3.shape

In [120]:
#getting the area per room
df4['area_per_room']=df4['total_sqft']/df4['size']
df4.head()

#### outlier removal process 1:
##### remove records having area per room less than 300

In [121]:

df5=df4[~(df4.area_per_room<300)]
df5.shape

#### outlier removal process 2:
##### we need to check and remove all the data pointers which are beyond 1st standerd deviation for all the location
##### which means we need to find describe based on location

In [122]:
df5.ppsqft.describe()#we can see there are extrems end in the data.

In [123]:
#removes the records which are beyond 1 std  from the mean based on the location
def remove_pps_outliers(df):
    df_out=pd.DataFrame()
    for key,subdf in df.groupby('location'):
        m=np.mean(subdf.ppsqft)
        std=np.std(subdf.ppsqft)
        reduced_df=subdf[(subdf.ppsqft>=(m-std)) & (subdf.ppsqft<=(m+std))]
        df_out=pd.concat([df_out,reduced_df],ignore_index=True)
    return  df_out
df6=remove_pps_outliers(df5)
df6.shape

In [124]:
df5.head()

In [125]:
df5.total_sqft

In [126]:
df5.describe()

#### outlier removal 3:

##### If the location and sqft is same  then the price of  2BHK should not be more than 3 BHK

In [127]:
# visualizing the condition 1st:

def viz_bhk(df,location):
    
    plt.figure(figsize=(10,5))
    #fig, ax = plt.subplots()
    #matplotlib.rcParams['figure.figsize']=(20,10)
    bhk2=df[(df['location']==location) &( df['size']==2)]
    bhk3=df[(df['location']==location) &( df['size']==3)]
    plt.scatter(bhk2.total_sqft,bhk2.price,color='blue',label='2 bhk',s=50)
    plt.scatter(bhk3.total_sqft,bhk3.price,color='green',label='3 bhk',marker="+",s=50)
    #plt.xticks(np.arange(min(bhk3.total_sqft), max(bhk3.total_sqft)+1, 1.0))
    plt.xlabel("toata sqft area")
    plt.ylabel("price per square feet")
    plt.title(location)
    plt.legend()
   
    
viz_bhk(df5,"Rajaji Nagar")

In [128]:
# removing the data points where price of 2 bhk is more than 3 BHK in the same area

In [129]:
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for size, bhk_df in location_df.groupby('size'):
            bhk_stats[size] = {
                'mean': np.mean(bhk_df.ppsqft),
                'std': np.std(bhk_df.ppsqft),
                'count': bhk_df.shape[0]
            }
        for size, bhk_df in location_df.groupby('size'):
            stats = bhk_stats.get(size-1)# gets the information for size-1 from above disctionary
            if stats and stats['count']>5:#only where the count of the size per location is grtr than 5
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.ppsqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')
df7 = remove_bhk_outliers(df6)
df7.shape

In [130]:
viz_bhk(df7,"Rajaji Nagar")

#### Outlier removal process 4: 

will check the number of bathrooms

In [131]:
df7.shape

In [132]:
df8=df7[~(df7['bath']>df7['size']+2)]

In [133]:
df8.shape

### Understanding the data:

##### checking the distribution

In [134]:
#Checking the distribution of the price
plt.figure(figsize=(10,5))
ax=sns.histplot(df7.ppsqft,bins=8)
#plt.hist(df7.ppsqft,rwidth=.8)
ax.set(xlabel="PPsqft", ylabel = "count")

In [135]:
#distribution of size 
plt.figure(figsize=(10,5))
sns.countplot(x=df7['size'].astype(str),order=df7['size'].astype(str).value_counts().index)

In [136]:
plt.figure(figsize=(6,3))
#sns.barplot(x=df.area_type,y=df.price)
sns.countplot(x=df7['area_type'].astype(str),order=df7['area_type'].astype(str).value_counts().index)

#### dropping unneccery columns:


In [137]:
df8.head()

In [138]:
df9=df8.join(pd.get_dummies(df8['location'], prefix='', prefix_sep=''))

In [139]:
df9.head()

In [140]:
df9.drop(columns=['balcony','ppsqft','area_per_room','other','area_type','location','availability'],inplace=True)

In [141]:
df9.head()

### model building

In [142]:
#Split the datset in X and y
x=df9.drop(columns=['price'],axis='columns')
y=(df9.price)


In [143]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y)

In [144]:
x.head()

In [145]:
from sklearn.linear_model import LinearRegression
model1=LinearRegression(n_jobs=1000)
model1.fit(x_train,y_train)
print('LR train model:' + str( model1.score(x_train,y_train)))
print('LR test model:' + str(model1.score(x_test,y_test)))

In [146]:
#K-fold

from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv=ShuffleSplit(n_splits=5,test_size=.25,random_state=0)

cross_val_score(LinearRegression(),x,y,cv=cv)

In [147]:
#Trying diffrent algorithm:Hyperparameters-gridsearchmethod
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge


def find_best_model(x,y):
    algos={
        
        'linear_regression' :{
        
        'model':LinearRegression(),
        'params':{
            'normalize':[True,False]
            
        }
    },

          'lasso' :{
        
        'model':Lasso(),
        'params':{
            'selection':['random','cyclic'],
            'alpha' :[1,2,.1,.001]            
        }
    },

          'Elastic' :{
        
        'model':ElasticNet(),
        'params':{
            #'selection':['random','cyclic'],
            'l1_ratio':[1,.001],
            'alpha' : [.1,.001,1,.01]       
        }
    },
     
          'deciscion_tree' :{
        
        'model':DecisionTreeRegressor(),
        'params':{
            'criterion':['mse','friedman_mse'],
            'splitter':['best','random']
            
        }
    }
          
    }
    
    scores=[]
    cv=ShuffleSplit(n_splits=5,test_size=.25,random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(x,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model(x,y)

In [148]:
#np.where(x.columns=='1st Block Jayanagar')[0][0]

In [149]:
df9.head()

In [150]:
x.shape

In [151]:
x_train.head()

In [152]:
def predict_price(size,total_sqft,bath,location):    
    loc_index = np.where(x.columns==location)[0][0]

    a = np.zeros(len(x.columns))
    a[0] = size
    a[1] = total_sqft
    a[2] = bath
    if loc_index >= 0:
        a[loc_index] = 1

    return model1.predict([a])[0]

In [153]:
#np.where(x.columns=='1st Phase JP Nagar')[0][0]

In [154]:
predict_price(2, 1000, 2,'1st Phase JP Nagar')

#### Export the tested model to a pickle file

In [156]:
import pickle
with open('banglore_home_prices_model.pickle','wb') as f:
    pickle.dump(model1,f)

#### Export location and column information to a file that will be useful later on in our prediction application

In [157]:
import json
columns = {
    'data_columns' : [col.lower() for col in x.columns]
}
with open("columns.json","w") as f: 
    f.write(json.dumps(columns))